<a href="https://colab.research.google.com/github/Summit2/ML_education/blob/main/Comparison_of_GD_methods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [88]:
import numpy as np

import seaborn as sns
sns.set(style="darkgrid", font_scale=1.4)
from matplotlib import pyplot as plt

from sklearn.model_selection import train_test_split

import torch
from torch import nn
from torch.nn import functional as F

from torch.utils.data import TensorDataset, DataLoader

from torchvision.datasets import MNIST
from torchvision import transforms
from tqdm.autonotebook import tqdm, trange

<ipython-input-88-a486a786c1f7>:17: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [56]:
data_transforms = transforms.Compose(
    [
     transforms.ToTensor(),
     transforms.Normalize((0.5),(0.5))

    ]

)

In [57]:
root = './'
train_dataset = MNIST(root, transform=data_transforms, download = True)
val_dataset = MNIST(root, transform=data_transforms,download = True)

train_dataloader = DataLoader(train_dataset, batch_size = 128)
val_dataloader =  DataLoader(val_dataset, batch_size = 128)

loaders = {'train' : train_dataloader,
           'valid' : val_dataloader }

In [83]:

class LeNet (nn.Module):
  def __init__(self):
      super(LeNet, self).__init__()

      self.conv1 = nn.Conv2d(1,6,3)

      self.pool1 = nn.MaxPool2d(2)
      self.conv2 = nn.Conv2d(6,16,3)


      self.fc1 = nn.Linear(1936,120)

      self.fc2 = nn.Linear(120,84)

      self.fc3 = nn.Linear(84,10)

      self.activation = nn.ReLU(inplace=True)

  def forward(self,x):
      x = self.activation(self.conv1(x))
      # print(x)
      x = self.pool1(x)
      x = self.activation(self.conv2(x))

      x = torch.flatten(x, start_dim = 1)
      x = self.activation(self.fc1(x))
      x = self.activation(self.fc2(x))
      x = self.fc3(x)

      return x



In [84]:
LeNet()
# train_loop(optimizer)

LeNet(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=1936, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
  (activation): ReLU(inplace=True)
)

In [90]:
# def train_loop(optimizer):

device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = LeNet()
model = model.to(device)
optimizer = torch.optim.SGD(params = model.parameters(), lr = 0.01)
criterion = torch.nn.CrossEntropyLoss()


max_epochs = 10
accuracy = {"train": [], "valid": []}



for epoch in range(max_epochs):
  for k, dataloader in loaders.items():
    epoch_correct = 0
    epoch_all = 0
    for x_batch, y_batch in dataloader:
      x_batch, y_batch = x_batch.to(device) ,y_batch.to(device)
      # print(type(x_batch))

      if k == "train":

          model.train()
          optimizer.zero_grad()
          # print((x_batch).size())
          outp = model(x_batch)

          loss = criterion(outp, y_batch)
          loss.backward()
          optimizer.step()


      elif k == "valid":
        model.eval()
        with torch.no_grad():
          outp = model(x_batch)

      #посчитаем accuracy
      predictions = outp.argmax(-1)
      # print(predictions)


      all = 128
      correct = (predictions.detach().numpy() == y_batch.cpu().detach().numpy()).sum()

      epoch_all +=all
      epoch_correct+=correct

      # print(correct_answers)
    if k == 'train':

        print(f"Epoch {epoch+1}")
        print(f"Accuracy = {epoch_correct/epoch_all}")


KeyboardInterrupt: ignored